In [2]:
import gym
import matplotlib.pyplot as plt
import numpy as np

In [3]:
env = gym.make("CartPole-v0")
observation = env.reset()

[2018-01-23 17:56:34,263] Making new env: CartPole-v0


In [4]:
print(vars(env.action_space))
print(vars(env.observation_space))

SOLVED_STEPS = 195 # openAI says that solving is averaging at least 195 steps

{'n': 2}
{'low': array([ -4.80000000e+00,  -3.40282347e+38,  -4.18879020e-01,
        -3.40282347e+38]), 'high': array([  4.80000000e+00,   3.40282347e+38,   4.18879020e-01,
         3.40282347e+38])}


In [5]:
class Agent(object):
    def __init__(self):
        return
        
    def choose_action(self, state):
        action = env.action_space.sample()
        return action
        
    def learn(self, state, reward):
        return

In [6]:
agent = Agent()

In [12]:
for i in range(10): # episodes
    timesteps = 0
    state = env.reset()
    for t in range(1, SOLVED_STEPS+1):
        env.render(close=False)
        action = agent.choose_action(state)
        state, reward, done, info = env.step(action)
        agent.learn(state, reward)
        if done:
            print("Episode {} finished after {} timesteps".format(i+1, t))
            break
    if t == SOLVED_STEPS:
        print("Episode {} was a success".format(i+1))

env.render(close=True)
env.reset()
env.close()

Episode 1 finished after 48 timesteps
Episode 2 finished after 15 timesteps
Episode 3 finished after 38 timesteps
Episode 4 finished after 27 timesteps
Episode 5 finished after 41 timesteps
Episode 6 finished after 30 timesteps
Episode 7 finished after 47 timesteps
Episode 8 finished after 15 timesteps
Episode 9 finished after 13 timesteps
Episode 10 finished after 21 timesteps
